In [19]:
import re
import pandas as pd
import numpy as np
import numpy as np

In [49]:
#load Excel file
df = pd.read_excel('./data/Passenger Immigration and Emigration List.xlsx', engine='openpyxl')


In [50]:
def data_process(input_string, debug=False):
    if pd.isna(input_string):
        return np.nan, np.nan, np.nan, np.nan, np.nan

    # 過濾不需要處理的特殊值
    skip_values = ['비자처리중', '비자 처리중 ', '축제전도착친구가픽업', '축제전 도착 (친구가픽업)']
    if input_string in skip_values:
        return np.nan, np.nan, np.nan, np.nan, np.nan

    # 提取韓文字符作為機場名
    airport = ''.join(filter(lambda x: '\uAC00' <= x <= '\uD7A3', input_string)).strip()

    # 分割日期和剩餘部分
    try:
        date, remain = input_string.split(airport)
    except ValueError:
        if debug:
            print(f"分割失敗：{input_string} (機場: {airport})")
        return np.nan, np.nan, np.nan, np.nan, np.nan

    # 日期處理
    try:
        date = re.sub(r'[/.]', '', date).strip()
        day_str = str(date[2:]).zfill(2)  # 填充日期至兩位數
        date = '11/' + day_str
    except Exception as e:
        if debug:
            print(f"日期處理失敗：{input_string} (日期: {date}, 錯誤: {e})")
        return np.nan, np.nan, np.nan, np.nan, np.nan

    # 處理航站樓部分
    ter_part = np.nan
    find_index = remain.find('T')
    if find_index != -1:
        ter_part = 'T' + remain[find_index + 1]
        remain = remain.split(ter_part)[-1].strip()

    # 處理時間部分
    time_part = np.nan
    find_index = remain.find(':')
    if find_index != -1:
        time_part = remain[find_index - 2:find_index + 3]
        remain = remain.split(time_part)[0].strip()

    # 提取航班號
    pattern = r'[A-Za-z0-9]+'  # 匹配所有英文字母和數字
    flight = re.findall(pattern, remain)
    flight = flight[0].strip() if flight else np.nan

    # Debug 輸出
    if debug:
        print(f"結果：日期={date}, 機場={airport}, 航站樓={ter_part}, 時間={time_part}, 航班號={flight}")

    return date, airport, ter_part, time_part, flight


In [51]:
status = 'departure'
# status = 'arrive'

# 將解析函數 data_process 的返回值分配到多列
# 假設 `status` 是一個變量，動態生成列名
df[['{} date'.format(status), '{} airport'.format(status), '{} ter_part'.format(status), '{} time_part'.format(status), '{} flight'.format(status)]] = df['{}'.format(status)].apply(
    lambda x: pd.Series(data_process(x))
)

# 刪除 `date` 或 `time_part` 為 NaN 的行，因為它們是後續處理的必要數據
df = df.dropna(subset=['{} date'.format(status)]).dropna(subset=['{} time_part'.format(status)])

# 去掉 `date` 和 `time_part` 列中可能存在的多餘空格
df['{} date'.format(status)] = df['{} date'.format(status)].str.strip()
df['{} time_part'.format(status)] = df['{} time_part'.format(status)].str.strip()

# 合併 `date` 和 `time_part` 列，並補充年份，生成完整的日期時間格式
# 假設目標年份是 2024
df['DateTime'] = pd.to_datetime(
    df['{} date'.format(status)] + f'/2024 ' + df['{} time_part'.format(status)], 
    format='%m/%d/%Y %H:%M'  # 指定日期和時間的格式
)

# 按 `DateTime` 列進行排序，確保數據按時間順序排列
df = df.sort_values(by='DateTime')


In [52]:
df

,Name,arrive,departure,departure date,departure airport,departure ter_part,departure time_part,departure flight
0,N1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N2,11.03/인천T1 UO618/11:40,11/10 인천T1 UO631/15:55,11/10,인천,T1,15:55,UO631
2,N3,11/3 인천T1-CX434/12:45,11/9 인천T1- CX419 19:35,11/09,인천,T1,19:35,CX419
3,N4,11/3 인천T1-CX434/12:45,11/9 인천T1- CX419 19:35,11/09,인천,T1,19:35,CX419
4,N5,11.03 / 인천T1 UO630/ 14:45,11.8 교통을 스스로 처리하다.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
135,N136,11.3 / 인천T2 -CI164 / 10:55,11.10 / 인천T2 - CI165 /12:00,11/10,인천,T2,12:00,CI165
136,N137,11.3 / 인천T2 -CI164 / 10:55,11.10 / 인천T2 - CI165 /12:00,11/10,인천,T2,12:00,CI165
137,N138,11.2 / 인천T2 - CI160 /11:20,11.10 / 인천T2 / 12:25,11/10,인천,T2,12:25,NaN
138,N139,11.2 / 인천T2 - CI160 /11:20,11.10 / 인천T2 / 12:25,11/10,인천,T2,12:25,NaN


In [54]:
# save to excel
df.to_excel('./result/{}.xlsx'.format(status), index=False)
